In [387]:
import torch
from models import MVCNN
from torchvision import transforms
import os
from PIL import Image
import re

In [388]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [389]:
svcnn = MVCNN.SVCNN(name="VGG11_SVCNN")
svcnn_weights = torch.load(
    "trained-models/MVCNN/model-00030.pth", weights_only=True, map_location=device
)
svcnn.load_state_dict(svcnn_weights, strict=False)
svcnn.eval()
svcnn.to(device)

# mvcnn = MVCNN.MVCNN(name="VGG11_MVCNN", model=svcnn)
# mvcnn_weights = torch.load(
#     "trained-models/MVCNN/model-00030.pth", weights_only=True, map_location=device
# )
# mvcnn.load_state_dict(mvcnn_weights, strict=False)
# mvcnn.eval()
# mvcnn.to(device)

SVCNN(
  (net_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [390]:
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

In [391]:
def init_csv():
    with open("svcnn-grads.csv", "w") as file:
        file.write("model,model_id,imp1,imp2,imp3,imp4,imp5,imp6,imp7,imp8,imp9,imp10,imp11,imp12\n")
    print("File Initialised")

In [392]:
def save_to_csv(grads: dict, model: str, model_id: int):
    with open ('svcnn-grads.csv', 'a') as file:
        file.write(f"{model},{model_id},")
        for i in range(0, 12):
            file.write(f"{grads[i]},")
        file.write("\n")
    print("File Updated")

In [393]:
def run_inference(svcnn, images_dir):
    outputs = []
    images = os.listdir(images_dir)
    images = sorted(
        images, key=lambda x: int(re.search(r'_(\d+)\.png', x)[1])
    )
    # images = [f"{images_dir}/{image}" for image in images]
    images = [os.path.join(images_dir, image) for image in images]
    svcnn = svcnn.to(device)
    svcnn.eval()
    # print(images)
    with torch.no_grad():
        for _ in range(12):
            image = Image.open(images[_])
            image = image.convert('RGB')
            image = transforms(image)
            image = image.to(device)
            image = image.unsqueeze(0)
            image = image.to(device)
            output = svcnn(image)
            outputs.append(output)
    return outputs

In [394]:
path ='../ModelNet40-12View/'
objects = [obj for obj in os.listdir(path) if not obj.startswith('.')] 
object_paths = [os.path.join(path, i, 'test') for i in objects]

model_paths = []
for obj_path in object_paths:
    if not os.path.isdir(obj_path):
        continue
    models_in_folder = [
        os.path.join(obj_path, fname)
        for fname in os.listdir(obj_path)
        if not fname.startswith('.')  # Exclude .DS_Store or other hidden files
    ]
    model_paths.extend(models_in_folder)

In [395]:
def rem_dup(arr):
    done = []
    for i in arr:
        if i not in done:
            done.append(i)
    return done

In [396]:
class_names=['airplane','bed','bench','bookshelf','bottle','bowl','car','chair',
                'cone','cup','curtain','door','flower_pot','glass_box',
                'guitar','keyboard','lamp','laptop','mantel', 'person','piano',
                'plant','radio','range_hood','sink','stairs',
                'stool','tent','toilet','tv_stand','vase','wardrobe','xbox']

classes = {class_names[i]: i for i in range(len(class_names))}

In [397]:
len(model_paths)

1860

In [398]:
accuracies = []

In [399]:
def run_this():
    print(f"{'Index':<10} {'Model':<20} {'Model Ex':<20} {'Correct':<10} {'Wrong':<10} {'Accuracy':<10}")
    for count, path in enumerate(model_paths):
        klass = path.split('/')[2]
        karrect, wrong = 0, 0
        outputs = run_inference(svcnn, path)
        for view_tensor in outputs:
            view = torch.argmax(view_tensor, dim=1).item()
            if view == classes[klass]:
                karrect += 1
            else:
                wrong += 1
        print(f"{count:<10} {path.split('/')[2]:<20} {path.split('/')[4]:<20} {karrect:<10} {wrong:<10} {karrect/(karrect+wrong)*100:.2f}%")
        accuracies.append(karrect/(karrect+wrong)*100)

    print(f"Mean Accuracy: {sum(accuracies)/len(accuracies):.2f}%")

In [400]:
def when_i_run_this_i_get_12_images(path):
    images = os.listdir(path)
    images = sorted(
        images, key=lambda x: int(re.search(r'_(\d+)\.png', x)[1])
    )
    for image in images:
        images[images.index(image)] = os.path.join(path, image)
    return images

In [401]:
def run_through_net1(image):
    image = Image.open(image)
    image = image.convert('RGB')
    image = transforms(image)
    image = image.to(device)
    image = image.unsqueeze(0)
    image = image.to(device)
    with torch.no_grad():
        output =  svcnn.net_1(image)
    return output


In [402]:
def make_it_for_net2(inputs):
    pooled_features, _ = torch.max(torch.stack(inputs), dim=0, keepdim=False)
    pooled_features = pooled_features.view(1, -1)
    pooled_features = pooled_features.to(device)
    return pooled_features

In [403]:
def run_through_net2(inputs):
    with torch.no_grad():
        output = svcnn.net_2(inputs)
    output = torch.argmax(output, dim=1).item()
    return output

In [404]:
def run_without_view(view_num, images):
    images = [image for i, image in enumerate(images) if i != view_num]
    outputs = [run_through_net1(image) for image in images]
    pooled_features = make_it_for_net2(outputs)
    return run_through_net2(pooled_features)

In [405]:
def write_view_csv(path, view_num, correct_without):
    with open('svcnn-grads.csv', 'a') as file:
        file.write(f'{path},{view_num},{correct_without}\n')


In [406]:
with open('svcnn-grads.csv', 'w') as file:
    file.write("model,removed-view,correct-1-wrong-0\n")
    print("File Initialised")

File Initialised


In [407]:
for path in model_paths:
    images = when_i_run_this_i_get_12_images(path)
    klass = path.split('/')[2]
    for view_num in range(12):
        print(f"Running without view {view_num} for {path.split('/')[2]}")
        output = run_without_view(view_num, images)
        if output == classes[klass]:
            print(f"Correctly classified {path.split('/')[2]} without view {view_num}")
            write_view_csv(path.split('/')[4], view_num, 1)
        else:
            print(f"Incorrectly classified {path.split('/')[2]} without view {view_num}")
            write_view_csv(path.split('/')[4], view_num, 0)

Running without view 0 for lamp
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_1.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_2.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_3.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_4.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_5.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_6.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_7.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_8.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_9.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_10.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_11.png
torch.Size([1, 25088])
Incorrectly classified lamp without view 0
Running without view 1 for lamp
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_0.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_2.png
../ModelNet40-12View/lamp/test/lamp_

Running without view 0 for lamp
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_1.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_2.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_3.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_4.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_5.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_6.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_7.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_8.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_9.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_10.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_11.png
torch.Size([1, 25088])
Incorrectly classified lamp without view 0
Running without view 1 for lamp
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_0.png
../ModelNet40-12View/lamp/test/lamp_0135/lamp_0135_shaded_2.png
../ModelNet40-12View/lamp/test/lamp_

KeyboardInterrupt: 

In [19]:
accs = [0.586466165413534, 0.593984962406015,0.599892588614393,0.596133190118153,0.590225563909774,0.591299677765843,0.597207303974221,0.594522019334049,0.588077336197637,0.601503759398496,0.59828141783029,0.593984962406015]
accs = list(accs)
accs

[0.586466165413534,
 0.593984962406015,
 0.599892588614393,
 0.596133190118153,
 0.590225563909774,
 0.591299677765843,
 0.597207303974221,
 0.594522019334049,
 0.588077336197637,
 0.601503759398496,
 0.59828141783029,
 0.593984962406015]

In [20]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
accs = np.array(accs).reshape(-1, 1)
accs = scaler.fit_transform(accs)
accs = accs.flatten()
accs = list(accs)
accs = [float(i) for i in accs]
accs = [1-i for i in accs]
for item in accs:
    print(f"{item}")

1.0
0.5000000000000071
0.1071428571428541
0.35714285714281857
0.7500000000000355
0.6785714285714448
0.2857142857142989
0.4642857142857366
0.892857142857153
0.0
0.2142857142857082
0.5000000000000071
